# Day 5 - Part 2: 단타 & 가치 전략

## 학습 목표
- RSI를 사용한 단타 전략 구현하기
- PBR을 사용한 가치 전략 구현하기
- 두 전략의 특징과 차이 이해하기

## 전략 개요
1. **단타 전략**: RSI(상대강도지수)로 과매수/과매도 판단
2. **가치 전략**: PBR(주가순자산비율)로 저평가 종목 선택

## Part A: RSI 단타 전략

### 1. RSI란?

**RSI (Relative Strength Index, 상대강도지수)**
- 0~100 사이의 값
- 30 이하: 과매도 (너무 많이 떨어짐 → 살까?)
- 70 이상: 과매수 (너무 많이 올랐음 → 팔까?)
- 보통 14일 기준으로 계산

In [ ]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

print("라이브러리 임포트 완료!")

### 2. 데이터 다운로드 (삼성전자 1개 종목)

In [ ]:
# 삼성전자 데이터
ticker = '005930.KS'
data = yf.download(ticker, start='2020-01-01', end='2023-12-31', progress=False)

print(f"삼성전자 데이터: {len(data)}일")
print(f"   기간: {data.index[0].date()} ~ {data.index[-1].date()}")

### 3. RSI 계산 함수

In [ ]:
def calculate_rsi(close_prices, period=14):
    """
    RSI 계산
    
    Parameters:
    - close_prices: 종가 Series
    - period: 기간 (기본 14일)
    
    Returns:
    - RSI Series (0~100)
    """
    # 가격 변화
    delta = close_prices.diff()
    
    # 상승/하락 분리
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # 평균 계산
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # RS = 평균 상승 / 평균 하락
    rs = avg_gain / avg_loss
    
    # RSI = 100 - (100 / (1 + RS))
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# RSI 계산
data['RSI'] = calculate_rsi(data['Close'], period=14)

# 결과 확인
print("\n최근 5일 RSI:")
print(data[['Close', 'RSI']].tail())

### 4. RSI 시각화

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 주가
axes[0].plot(data.index, data['Close'], label='종가', linewidth=1.5)
axes[0].set_title('삼성전자 주가', fontsize=14, fontweight='bold')
axes[0].set_ylabel('가격 (원)', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# RSI
axes[1].plot(data.index, data['RSI'], label='RSI', color='purple', linewidth=1.5)
axes[1].axhline(y=70, color='red', linestyle='--', label='과매수 (70)', linewidth=1)
axes[1].axhline(y=30, color='green', linestyle='--', label='과매도 (30)', linewidth=1)
axes[1].fill_between(data.index, 70, 100, alpha=0.1, color='red')
axes[1].fill_between(data.index, 0, 30, alpha=0.1, color='green')
axes[1].set_title('RSI (14일)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('RSI', fontsize=12)
axes[1].set_xlabel('날짜', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 5. RSI 단타 전략 구현

In [ ]:
class RSIStrategy(bt.Strategy):
    """
    RSI 단타 전략
    - RSI < 30: 매수 (과매도)
    - RSI > 70: 매도 (과매수)
    """
    
    params = (
        ('rsi_period', 14),
        ('rsi_low', 30),   # 매수 시그널
        ('rsi_high', 70),  # 매도 시그널
    )
    
    def __init__(self):
        # RSI 지표 계산 (Backtrader 내장 함수 사용)
        self.rsi = bt.indicators.RSI(
            self.data.close,
            period=self.params.rsi_period
        )
        
        self.order = None
        self.buy_signals = 0
        self.sell_signals = 0
        
        print(f"RSI 전략 초기화 (기간: {self.params.rsi_period}일)")
    
    def next(self):
        """매일 실행"""
        
        # 이미 주문 중이면 대기
        if self.order:
            return
        
        current_date = self.data.datetime.date(0)
        
        # RSI < 30: 매수 신호
        if self.rsi[0] < self.params.rsi_low:
            if not self.position:  # 포지션 없으면
                # 전체 금액의 50%로 매수
                size = int((self.broker.getvalue() * 0.5) / self.data.close[0])
                self.order = self.buy(size=size)
                self.buy_signals += 1
                print(f"{current_date}: 매수 신호 (RSI={self.rsi[0]:.1f})")
        
        # RSI > 70: 매도 신호
        elif self.rsi[0] > self.params.rsi_high:
            if self.position:  # 포지션 있으면
                self.order = self.close()  # 전량 매도
                self.sell_signals += 1
                print(f"{current_date}: 매도 신호 (RSI={self.rsi[0]:.1f})")
    
    def notify_order(self, order):
        """주문 체결 알림"""
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"   매수 체결: {order.executed.price:,.0f}원")
            else:
                print(f"   매도 체결: {order.executed.price:,.0f}원")
            
            self.order = None
    
    def stop(self):
        """백테스트 종료 시"""
        print(f"\n거래 요약:")
        print(f"   매수 신호: {self.buy_signals}회")
        print(f"   매도 신호: {self.sell_signals}회")

### 6. RSI 전략 백테스트

In [ ]:
# Cerebro 생성
cerebro = bt.Cerebro()

# 초기 자금 및 수수료
cerebro.broker.setcash(100_000_000)
cerebro.broker.setcommission(commission=0.001)

# 데이터 추가
data_bt = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_bt)

# 전략 추가
cerebro.addstrategy(RSIStrategy, rsi_period=14, rsi_low=30, rsi_high=70)

# 분석기 추가
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

print(f"\n{'='*60}")
print(f"RSI 단타 전략 백테스트 시작!")
print(f"{'='*60}\n")

# 실행
start_value = cerebro.broker.getvalue()
results = cerebro.run()
strat = results[0]
end_value = cerebro.broker.getvalue()

# 결과
profit = end_value - start_value
return_pct = (profit / start_value) * 100

returns_analysis = strat.analyzers.returns.get_analysis()
dd_analysis = strat.analyzers.drawdown.get_analysis()
trades_analysis = strat.analyzers.trades.get_analysis()

print(f"\n{'='*60}")
print(f"RSI 전략 결과")
print(f"{'='*60}")
print(f"시작 자금: {start_value:,.0f}원")
print(f"최종 자금: {end_value:,.0f}원")
print(f"수익금: {profit:,.0f}원")
print(f"수익률: {return_pct:.2f}%")
print(f"최대 낙폭: {dd_analysis['max']['drawdown']:.2f}%")
print(f"{'='*60}\n")

# 포트폴리오 그래프
cerebro.plot(style='candlestick')
plt.show()

---
## Part B: PBR 가치 전략

### 1. PBR이란?

**PBR (Price to Book Ratio, 주가순자산비율)**
- PBR = 주가 / 주당순자산(BPS)
- PBR < 1: 회사 자산보다 주가가 낮음 (저평가?)
- PBR이 낮을수록 "싸다"고 판단
- 가치투자자들이 애용하는 지표

### 2. 데이터 다운로드 (10개 종목)

In [ ]:
# 가치주 분석용 종목들
value_tickers = {
    '055550.KS': '신한지주',
    '086790.KS': '하나금융지주',
    '105560.KS': 'KB금융',
    '005380.KS': '현대차',
    '000270.KS': '기아',
    '012330.KS': '현대모비스',
    '009540.KS': 'HD현대중공업',
    '010950.KS': 'S-Oil',
    '011170.KS': '롯데케미칼',
    '096770.KS': 'SK이노베이션'
}

value_data = {}

print("가치주 데이터 다운로드...\n")

for ticker, name in value_tickers.items():
    try:
        data = yf.download(ticker, start='2020-01-01', end='2023-12-31', progress=False)
        if not data.empty:
            value_data[ticker] = data
            print(f"{name}: {len(data)}일")
    except:
        print(f"{name}: 오류")

print(f"\n{len(value_data)}개 종목 준비 완료")

### 3. PBR 데이터 생성 (시뮬레이션)

실제로는 재무제표에서 가져와야 하지만, 교육용으로 간단히 시뮬레이션합니다.

In [ ]:
# PBR 데이터 생성 (교육용 - 실제로는 FinanceDataReader 등 사용)
np.random.seed(42)

pbr_data = {}

for ticker, name in value_tickers.items():
    # 각 종목에 고유한 PBR 할당 (0.3 ~ 1.5 범위)
    base_pbr = np.random.uniform(0.3, 1.5)
    # 시간에 따라 약간씩 변동
    pbr_data[ticker] = {
        'name': name,
        'pbr': base_pbr,
        'base_value': 'low' if base_pbr < 0.7 else 'medium' if base_pbr < 1.0 else 'high'
    }

# PBR 순위
sorted_pbr = sorted(pbr_data.items(), key=lambda x: x[1]['pbr'])

print("PBR 순위 (낮을수록 저평가):\n")
for i, (ticker, info) in enumerate(sorted_pbr, 1):
    print(f"{i:2d}. {info['name']:15s}: PBR={info['pbr']:.2f} ({info['base_value']})")

### 4. 가치 전략 구현

In [ ]:
class ValueStrategy(bt.Strategy):
    """
    가치 전략
    - PBR 하위 N개 종목 선택
    - 1년에 한 번 리밸런싱
    """
    
    params = (
        ('pbr_dict', None),  # {ticker: pbr} 딕셔너리
        ('top_n', 5),
        ('rebalance_dates', None),
    )
    
    def __init__(self):
        self.rebalance_dates = self.params.rebalance_dates
        self.next_rebalance_idx = 0
        self.rebalance_count = 0
        
        print(f"가치 전략 초기화 (상위 {self.params.top_n}개 선택)")
    
    def next(self):
        """매일 실행"""
        current_date = pd.Timestamp(self.datas[0].datetime.date(0))
        
        if self.next_rebalance_idx < len(self.rebalance_dates):
            next_rebalance = self.rebalance_dates[self.next_rebalance_idx]
            
            if current_date >= next_rebalance:
                self.rebalance()
                self.next_rebalance_idx += 1
    
    def rebalance(self):
        """리밸런싱"""
        self.rebalance_count += 1
        current_date = self.datas[0].datetime.date(0)
        
        # PBR 기준 정렬
        pbr_ranking = []
        for d in self.datas:
            ticker = d._name
            if ticker in self.params.pbr_dict:
                pbr_ranking.append((d, self.params.pbr_dict[ticker]['pbr']))
        
        # PBR 낮은 순으로 정렬
        pbr_ranking.sort(key=lambda x: x[1])
        top_n = [item[0] for item in pbr_ranking[:self.params.top_n]]
        
        # 기존 포지션 정리
        for d in self.datas:
            if self.getposition(d).size > 0 and d not in top_n:
                self.close(d)
        
        # 동일 가중 투자
        target_value = self.broker.getvalue() / self.params.top_n
        
        for d in top_n:
            target_shares = int(target_value / d.close[0])
            diff = target_shares - self.getposition(d).size
            
            if diff > 0:
                self.buy(data=d, size=diff)
            elif diff < 0:
                self.sell(data=d, size=-diff)
        
        # 로그
        print(f"\n{'='*60}")
        print(f"{current_date} (리밸런싱 #{self.rebalance_count})")
        print(f"포트폴리오 가치: {self.broker.getvalue():,.0f}원")
        print(f"\n선택된 종목 (PBR 하위 {self.params.top_n}개):")
        for i, (d, pbr) in enumerate(pbr_ranking[:self.params.top_n], 1):
            name = self.params.pbr_dict[d._name]['name']
            print(f"  {i}. {name}: PBR={pbr:.2f}")
        print(f"{'='*60}")

### 5. 연간 리밸런싱 날짜 설정

In [ ]:
# 매년 1월 첫 거래일
annual_dates = []
for year in range(2020, 2024):
    # 각 연도 1월 데이터에서 첫 거래일 찾기
    first_ticker = list(value_data.keys())[0]
    jan_data = value_data[first_ticker][f'{year}-01']
    if len(jan_data) > 0:
        annual_dates.append(jan_data.index[0])

print(f"연간 리밸런싱 날짜: {len(annual_dates)}회")
for date in annual_dates:
    print(f"   - {date.strftime('%Y-%m-%d')}")

### 6. 가치 전략 백테스트

In [ ]:
# Cerebro 생성
cerebro2 = bt.Cerebro()

# 초기 자금 및 수수료
cerebro2.broker.setcash(100_000_000)
cerebro2.broker.setcommission(commission=0.001)

# 데이터 추가
for ticker, data in value_data.items():
    data_bt = bt.feeds.PandasData(
        dataname=data,
        name=ticker,
        fromdate=datetime(2020, 1, 1),
        todate=datetime(2023, 12, 31)
    )
    cerebro2.adddata(data_bt)

# 전략 추가
cerebro2.addstrategy(
    ValueStrategy,
    pbr_dict=pbr_data,
    top_n=5,
    rebalance_dates=annual_dates
)

# 분석기
cerebro2.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro2.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

print(f"\n{'='*60}")
print(f"가치 전략 백테스트 시작!")
print(f"{'='*60}\n")

# 실행
start_value2 = cerebro2.broker.getvalue()
results2 = cerebro2.run()
strat2 = results2[0]
end_value2 = cerebro2.broker.getvalue()

# 결과
profit2 = end_value2 - start_value2
return_pct2 = (profit2 / start_value2) * 100

returns2 = strat2.analyzers.returns.get_analysis()
dd2 = strat2.analyzers.drawdown.get_analysis()

print(f"\n{'='*60}")
print(f"가치 전략 결과")
print(f"{'='*60}")
print(f"시작 자금: {start_value2:,.0f}원")
print(f"최종 자금: {end_value2:,.0f}원")
print(f"수익금: {profit2:,.0f}원")
print(f"수익률: {return_pct2:.2f}%")
print(f"최대 낙폭: {dd2['max']['drawdown']:.2f}%")
print(f"{'='*60}\n")

# 그래프
cerebro2.plot(style='candlestick')
plt.show()

## 비교 분석

In [ ]:
print(f"\n{'='*70}")
print(f"전략 비교")
print(f"{'='*70}")
print(f"{'항목':<20} {'RSI 단타':>20} {'PBR 가치':>20}")
print(f"{'─'*70}")
print(f"{'총 수익률':<20} {return_pct:>19.2f}% {return_pct2:>19.2f}%")
print(f"{'최대 낙폭':<20} {dd_analysis['max']['drawdown']:>19.2f}% {dd2['max']['drawdown']:>19.2f}%")
print(f"{'리밸런싱 횟수':<20} {'자주':>20} {'연 1회':>20}")
print(f"{'투자 기간':<20} {'단기 (일~주)':>20} {'장기 (년)':>20}")
print(f"{'='*70}\n")

## 정리

**RSI 단타 전략**:
- 단기 과열/과냉 구간 포착
- 거래 빈도 높음
- 수수료 영향 큼
- 변동성 장세에 유리

**PBR 가치 전략**:
- 저평가 종목 장기 보유
- 거래 빈도 낮음
- 수수료 영향 작음
- 안정적 장세에 유리

**핵심 차이**:
- 투자 기간: 단기 vs 장기
- 리밸런싱: 자주 vs 드물게
- 수수료: 높음 vs 낮음

다음 시간: DL로 모멘텀 전략을 개선해봅시다!